In [1]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from uuid import uuid4
from dotenv import load_dotenv
load_dotenv()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT_ID = os.getenv('AZURE_OPENAI_DEPLOYMENT_ID')
AZURE_OPENAI_KEY = os.getenv('AZURE_OPENAI_KEY')
AZURE_API_VERSION = os.getenv('AZURE_API_VERSION')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)

In [3]:
llm = AzureChatOpenAI(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            azure_deployment=AZURE_OPENAI_DEPLOYMENT_ID,
            api_version=AZURE_API_VERSION,
            api_key=AZURE_OPENAI_KEY,
            temperature=0.0,
            verbose=True,
        )

embedding_llm = AzureOpenAIEmbeddings(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            azure_deployment='embedding-ada-crayon',
            api_key=AZURE_OPENAI_KEY,
            api_version=AZURE_API_VERSION,
        )

In [7]:
loader = PyPDFLoader(r"Dummy - CB Policy.pdf")
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
splits = text_splitter.split_documents(pages)

In [8]:
splits

[Document(metadata={'source': 'Dummy - CB Policy.pdf', 'page': 0}, page_content='1 \n \nOCBC Information Classification: Internal   \n \n \n \n \n \n \n \n \nCompensation & Benefits \nPolicy  \nPT CRAYON SHINCHAN  \n \nPolicy Effective Date: < 07/08/24> \n \n \n \n \n \n \nNo part of this documentation may be reproduced or transmitted in any form or by any means, electronic \nor mechanical, including photocopying or recording, for any purpose without express written permission \nof the CEO of PT CRAYON  SHINCHAN.  \n \n© 2021, <Company Name Here>. All Rights Reserved'),
 Document(metadata={'source': 'Dummy - CB Policy.pdf', 'page': 1}, page_content='2 \n \nOCBC Information Classification: Internal   \nRevision History  \n  \nVer \nNo. Change \nDescription  Prepared \nBy Reviewed By  Approved \nBy Date  \n      \n  \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nTABLE OF CONTENTS  \n \n \n \n \n \n \nObjective  4'),
 Document(metadata={'so

In [9]:
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "ocbc-hr-gpt"  # change if desired
index = pc.Index(index_name)
llm = AzureChatOpenAI(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            azure_deployment=AZURE_OPENAI_DEPLOYMENT_ID,
            api_version=AZURE_API_VERSION,
            api_key=AZURE_OPENAI_KEY,
            temperature=0.0,
            verbose=True,
        )

embedding_llm = AzureOpenAIEmbeddings(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            azure_deployment='embedding-ada-crayon',
            api_key=AZURE_OPENAI_KEY,
            api_version=AZURE_API_VERSION,
        )

vector_store = PineconeVectorStore(index=index, embedding=embedding_llm)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={'k': 6})

In [22]:
from langchain_core.prompts import PromptTemplate
from templates.prompt import QA_PROMPT


prompt = PromptTemplate.from_template(QA_PROMPT)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()} | RunnablePassthrough()

)


In [23]:
res = rag_chain.invoke("what are the mandatory benefits given?")

In [24]:
res['context']

"● Basic Salary  \n● Housing Allowance  \nThe total of Basic Salary and Housing Allowance is the Gross salary.  \n● The Basic Salary is 50% of the Gross salary.  \nAllowances  \n \nAny other additi onal benefits, i.e. Car Allowance + Fuel Allowance (depending \nupon the hierarchy within the organisation and job responsibilities)  \nThe following are the Variable Benefits of the compensation Structure  \n● Overtime (depending upon the hierarchy within the organ isation)  \n● Incentives and /or Bonus (depending upon the hierarchy within the \norganisation and job within the organisation)  \n Mandatory Benefits  \n \nThe following are the components of Mandatory Benefits that are extended to \nemployees.  \n● Visa and Employment processing fee  – The visa fee of employees \nshall be borne by the organisation. However, the visa for immediate \nfamily members  shall have to be borne by employees themselves. If \nany requests for assistance in providing family visa is made by\n\norganisation